# Model Training

If you have your own ground truth energy data, you can train a custom RouteE powertrain model.

You'll want to make sure you've installed the proper dependencies that are not installed by default when you do a pip install. 

In this example, we'll use the scikit-learn based estimators which you can install by doing:

```bash
pip install nrel.routee.powertrain[scikit]
```

In [1]:
import nrel.routee.powertrain as pt

from nrel.routee.powertrain.trainers.sklearn_random_forest import SklearnRandomForestTrainer

For demonstration purposes, we'll use a very small set of training data:

In [2]:
import pandas as pd

df = pd.read_csv("../tests/routee-powertrain-test-data/sample_train_data.csv")

In [3]:
df.head()

,speed_mph,grade_dec,miles,gallons_fastsim,trip_id,road_class
0,7.632068,-0.008963,0.015469,0.000813,1,3
1,6.329613,-0.047001,0.003516,0.000149,1,3
2,12.248512,0.000000,0.003402,0.000074,1,4
3,23.752604,-0.000463,0.019768,0.002194,1,1
4,46.024926,-0.004641,0.038378,0.000970,1,0


This dataframe represents a set of road network links (i.e. roads) in which we've already computed the energy consumption over. In this case, we've use the Fastsim software to simulate a vehicle driving over a high resolution drive cycle and then have aggregated everything up to the link level. We also have link level attributes like average driving speed in mile per hour (`speed`), road gradient as a decimal (`grade`), road distance in miles (`miles`) and road classification as a integer category (`road_class`). Lastly, we have a trip identifier column (`trip_id`) which is only 1 in this case, represeting a single trip taken by this vehicle.

Ok, onto setting up the training pipeline.

First, we need to tell the trainer what feature sets we want to use for the internal estimators (Random Forests in this case). We can provide one or many feature sets, depending on the different features we might expect to see when apply this model. In this case, we'll just use three different features sets. One with just `speed`, one with `speed` and `grade` and then another with `speed`, `grade`, and `road_class`. This will make it such that our model is flexible to cases where we might only have speed information for a link or we might have more feature resolution.  

In [10]:
feature_set_1 = [pt.DataColumn(name="speed_mph", units="mph")]
feature_set_2 = [
    pt.DataColumn(name="speed_mph", units="mph"),
    pt.DataColumn(name="grade_dec", units="decimal")
]
feature_set_3 = [
    pt.DataColumn(name="speed_mph", units="mph"),
    pt.DataColumn(name="grade_dec", units="decimal"),
    pt.DataColumn(name="road_class", units="category")
]
features = [
    feature_set_1,
    feature_set_2,
    feature_set_3
]

Note that we didn't incude the distance column in any of our feature sets. That is because, RouteE Powertrain always requires distance information and so we have a special designation for distance in the training configuation whereas features can be any arbitrary link attribute. So, let's define our distance columns

In [11]:
distance = pt.DataColumn(name="miles", units="miles")

Now, we need to define our energy target which is gallons of gasoline simualted by Fastsim:

In [12]:
energy_target = pt.DataColumn(
    name="gallons_fastsim", 
    units="gallons_gasoline", 
)

Finally, we can build a model configuration that we can pass to the trainer. This will also include things like the vehicle powertrain type and a model name

In [13]:
config = pt.ModelConfig(
    vehicle_description="Test Vehicle",
    powertrain_type=pt.PowertrainType.ICE,
    feature_sets=features,
    distance=distance,
    target=energy_target,
    test_size=0.2
)

Now we build the random forest trainer and give it the desired parameters

In [14]:
trainer = SklearnRandomForestTrainer(
    max_depth=10,
    min_samples_split=10,
    n_estimators=20,
    cores=4
)

All trainers have a `train` method on them which will return a trained vehicle model

In [15]:
test_vehicle = trainer.train(df, config)

2023-10-23 16:05:56,833 [INFO] - filtered out 0 rows with energy rates outside of the limits of -inf and inf for energy target gallons_fastsim


With the model trained, we can inspect the errors for each estimator type and energy target (note, it's possible that we could have given multiple energy targets to the trainer, like gasoline and electricity for a plug-in hybrid vehicle)

In [16]:
test_vehicle.errors

====================================================
Feature Set ID:                  speed_mph
Target:                          gallons_fastsim
Link RMSE                        0.002
Link Norm RMSE                   1.026
Link Weighted RPD                0.850
Net Error                        -0.346
Actual Dist/Energy               18.872
Predicted Dist/Energy            28.876
Real World Predicted Dist/Energy 24.765
Trip RPD                         0.419
Trip Weighted RPD                0.419
Trip RMSE                        0.014
Trip Norm RMSE                   0.346
====================================================
Feature Set ID:                  grade_dec&speed_mph
Target:                          gallons_fastsim
Link RMSE                        0.001
Link Norm RMSE                   0.870
Link Weighted RPD                0.614
Net Error                        -0.165
Actual Dist/Energy               18.872
Predicted Dist/Energy            22.591
Real World Predicted Dist/Energy 19.375
Trip RPD                         0.179
Trip Weighted RPD                0.179
Trip RMSE                        0.007
Trip Norm RMSE                   0.165
====================================================
Feature Set ID:                  grade_dec&road_class&speed_mph
Target:                          gallons_fastsim
Link RMSE                        0.001
Link Norm RMSE                   0.874
Link Weighted RPD                0.601
Net Error                        -0.150
Actual Dist/Energy               18.872
Predicted Dist/Energy            22.201
Real World Predicted Dist/Energy 19.040
Trip RPD                         0.162
Trip Weighted RPD                0.162
Trip RMSE                        0.006
Trip Norm RMSE                   0.150
====================================================

While this training dataset is far too small to draw real conclusions, these metrics can give you an idea of how well the model performed on a holdout test set (20% of the training data as we specificed by the `test_size` parameter in the configuration. 

Now, we can write the model to a json file that can be loaded later:

```python
test_vehicle.to_file("Test_Vehicle.json")
```